In [5]:
import re
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akshi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

In [7]:
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [4]:
out_df.text = out_df.text.apply(lambda x: preprocess(x))

NameError: name 'out_df' is not defined

In [8]:
import xml.etree.ElementTree as et 
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
stop_words = set(stopwords.words('english'))
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.parse.corenlp import CoreNLPDependencyParser
parser = CoreNLPDependencyParser()

In [9]:
def create_dataset(filename):
    xtree = et.parse(filename)
    xroot = xtree.getroot()
    
    d = {"id":[], "text":[], "term":[], "pol":[], "from":[], "to":[], "cat":[], "catpol":[]}
    
    for node in xroot:
        r_id = node.attrib.get("id")
        r_text = node.find("text").text if node is not None else None
        d["id"].append(r_id)
        d["text"].append(r_text)
    
    lis=[[],[],[],[]]
    lis2=[[],[]]
    
    for i in xroot.findall('sentence'):
        #for aspectTerms
        if len(i.findall('aspectTerms'))==0:
            d['term'].append(['#'])
            d['pol'].append(['#'])
            d['from'].append(['#'])
            d['to'].append(['#'])
        
        else:
            for j in i.findall('aspectTerms'):
                for k in j.findall('aspectTerm'):
                    d2 = k.attrib
                    lis[0].append(d2['term'])
                    lis[1].append(d2['polarity'])
                    lis[2].append(d2['from'])
                    lis[3].append(d2['to'])
                d['term'].append(lis[0])
                d['pol'].append(lis[1])
                d['from'].append(lis[2])
                d['to'].append(lis[3])
                lis=[[],[],[],[]]
    
        #for aspectCategories
        if len(i.findall('aspectCategories'))==0:
            d['cat'].append(['#'])
            d['catpol'].appenda(['#'])
        
        else:
            for j in i.findall('aspectCategories'):
                for k in j.findall('aspectCategory'):
                    d2 = k.attrib
                    lis2[0].append(d2['category'])
                    lis2[1].append(d2['polarity'])
                d['cat'].append(lis2[0])
                d['catpol'].append(lis2[1])
                lis2=[[],[]]
                
    out_df =  pd.DataFrame.from_dict(d)
    out_df.to_csv("Output.csv")
    return(out_df)

In [10]:
def Stop_word_remove(dataFrame):
    l=[]
    for sentence in dataFrame['text']:
        words = word_tokenize(sentence)
        #print(words)
        l1=[]
        for w in words:
            if w not in stop_words:
                l1.append(w)
        l.append(l1)
    dataFrame['filter'] = l

In [11]:
def lammitize(dataFrame):
    l=[]
    for sentence in dataFrame['filter']:
        l1=[]
        for word in sentence:
            l1.append(lemmatizer.lemmatize(word))
        l.append(l1)
    dataFrame['lemmatize'] = l

In [12]:
def pos_tag(dataFrame):
    l=[]
    for i in dataFrame['lemmatize']:
        l.append(nltk.pos_tag(i))
    dataFrame['pos_tag'] = l

In [13]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\akshi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [14]:
df = create_dataset("Restaurants_train.xml")
Stop_word_remove(df)
lammitize(df)
pos_tag(df)

In [15]:
df.to_csv("transform.csv")

In [20]:
def depen_parse(dataFrame):
    l=[]
    for i in dataFrame['text']:
        parse = next(parser.raw_parse(i))
        l.append(parse)
    dataFrame['dep_parse'] = l

In [21]:
depen_parse(df)

In [1]:
import quandl
api_key = 'andvh_sPuDBU7oJVN6pu'
newdf = quandl.get("WIKI/GOOGL", authtoken = api_key)
newdf.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [2]:
newdf = newdf[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]
newdf['HL_PCT'] = (newdf['Adj. High']-newdf['Adj. Low'])/newdf['Adj. Close']*100
newdf['PCT_Change']=(newdf['Adj. Close']-newdf['Adj. Open'])/newdf['Adj. Open']*100
newdf = newdf[['Adj. Close','HL_PCT','PCT_Change','Adj. Volume']]

In [3]:
newdf.head()

,Adj. Close,HL_PCT,PCT_Change,Adj. Volume
Date,,,,
2004-08-19,50.322842,8.072956,0.324968,44659000.0
2004-08-20,54.322689,7.921706,7.227007,22834300.0
2004-08-23,54.869377,4.049360,-1.227880,18256100.0
2004-08-24,52.597363,7.657099,-5.726357,15247300.0
2004-08-25,53.164113,3.886792,1.183658,9188600.0


In [19]:
import os
from nltk.parse.corenlp import CoreNLPServer
STANFORD = os.path.join("stanford-corenlp-full-2018-10-05")
server = CoreNLPServer(
   os.path.join(STANFORD, "stanford-corenlp-3.9.2.jar"),
   os.path.join(STANFORD, "stanford-corenlp-3.9.2-models.jar"),    
)
server.start()